# Gemma

<p align="center">
  <img src="https://ucarecdn.com/7b36315f-fc63-4f57-8344-12f1a993376f/" alt="arch" width="1000" height="165">
</p>

Gemma 1 вышла в феврале 2024 года.

По архитектуре модель больше всего похожа на Llama'у. Содержит уже знакомые нам RoPE и RMSNorm. Но есть и новинки:

* **Multi-Query Attention (MQA)** — крайне экономный вариант механизма внимания.
* **GeGLU** — гибридная функция активации. Почти клон SwiGLU :)

Обе довольно легкие для внедрения, по сравнению с прошлыми новинками :)


# Multi-Query Attention

По своей сути, Multi-Query Attention (MQA) — это частный случай Grouped Query Attention (GQA), который мы реализовали в уроке про Mistral.

<p align="center">
  <img src="https://ucarecdn.com/d1b0b294-9c02-4de1-9074-ae8ebfe96f6e/" alt="mqa" width="1000" height="217">
</p>

В GQA на каждую голову приходится один вектор запроса (query). При этом каждый вектор ключа (key) и значения (value) обслуживает ( n )-голов.
Так вот, в MQA на все головы (в одном блоке декодера) приходится всего по одному вектору ключа (key) и одному вектору значения (value). Это такая радикальная форма экономии :)


**Multi-Query Attention (разработка)**

In [21]:
import torch
from torch import nn
from typing import Optional


class RoPE(nn.Module):

    def __init__(self, head_size: int, max_seq_len: int, base: int = 10_000):
        super().__init__()
        assert head_size % 2 == 0, "head_size должен быть четным"

        # Вычисление частот: θ_i = base^(-2i/d) для i ∈ [0, d/2-1]
        freqs = 1.0 / (base ** (2 * torch.arange(head_size // 2).float() / head_size))

        # Позиции от 0 до max_seq_len-1
        positions = torch.arange(max_seq_len).float()

        # Внешнее произведение: m * θ_i для всех позиций и частот
        freq_matrix = positions.unsqueeze(1) * freqs.unsqueeze(0)

        # Предвычисление матриц косинусов и синусов
        self.register_buffer("cos_matrix", torch.cos(freq_matrix))
        self.register_buffer("sin_matrix", torch.sin(freq_matrix))

    def forward(self, x: torch.Tensor) -> torch.Tensor: # [batch_size × seq_len × head_size] [batch_size × num_heads × seq_len × head_size]
        batch_size, num_heads, seq_len, head_size = x.shape

        # Берем нужную часть матриц и приводим к типу x
        cos = self.cos_matrix[:seq_len].to(x.dtype)  # [seq_len, head_size//2]
        sin = self.sin_matrix[:seq_len].to(x.dtype)  # [seq_len, head_size//2]

        # Явное изменение формы для broadcasting
        cos = cos.reshape(1, 1, seq_len, head_size // 2)
        sin = sin.reshape(1, 1, seq_len, head_size // 2)

        # Разделяем на четные и нечетные компоненты по ПОСЛЕДНЕМУ измерению
        x_even = x[..., 0::2]  # [batch_size, num_heads, seq_len, head_size//2]
        x_odd = x[..., 1::2]   # [batch_size, num_heads, seq_len, head_size//2]

        # Применяем поворот: q' = q * cos(mθ) + rotate(q) * sin(mθ)
        x_rotated_even = x_even * cos - x_odd * sin
        x_rotated_odd = x_even * sin + x_odd * cos

        # Объединяем обратно в исходную размерность
        x_rotated = torch.stack([x_rotated_even, x_rotated_odd], dim=-1)
        x_rotated = x_rotated.flatten(-2)  # [batch_size, seq_len, head_size]

        return x_rotated

In [34]:
import torch
from torch import nn
import torch.nn.functional as F

class MultiQueryAttention(nn.Module):
    def __init__(
        self,
        num_q_heads: int,
        emb_size: int,
        head_size: int,
        max_seq_len: int,
        rope: RoPE = None,
        dropout: float = 0.1,
    ):
        super().__init__()
        self._num_q_heads = num_q_heads
        self._head_size = head_size
        self._max_seq_len = max_seq_len
        self._rope = rope
        
        self._q = nn.Linear(emb_size, num_q_heads * head_size)
        self._k = nn.Linear(emb_size,  head_size)
        self._v = nn.Linear(emb_size,  head_size)

        # Создание causal маски
        mask = torch.tril(torch.ones(max_seq_len, max_seq_len))
        self.register_buffer(
            "_tril_mask", mask.bool() if hasattr(torch, "bool") else mask.byte()
        )
        
        self._layer = nn.Linear(num_q_heads * head_size, emb_size)
        self._dropout = nn.Dropout(dropout)

    def forward(
        self,
        x: torch.Tensor,
        mask: torch.Tensor = None,
        use_cache: bool = True,
        cache: list = None,
    ):
        batch_size, seq_len, emb_size = x.shape
        if seq_len > self._max_seq_len:
            raise ValueError(
                f"Длина последовательности {seq_len} превышает максимум {self._max_seq_len}"
            )

        # Пропустите тензор x через матрицы Wq, Wk , Wv, чтобы получить матрицы запроса, ключа и значения.
        k = self._k(x)  # [B, T, hs]
        q = self._q(x)  # [B, T, hs]
        v = self._v(x)  # [B, T, hs]

        # Шаг 2: Изменение формы для multi-head
        # [batch_size, seq_len, num_heads * head_size] 
        # -> [batch_size, seq_len, num_heads, head_size]
        q = q.reshape(batch_size, seq_len, self._num_q_heads, self._head_size)
        k = k.reshape(batch_size, seq_len, 1, self._head_size)
        v = v.reshape(batch_size, seq_len, 1, self._head_size)

        # 3. Transpose: [B, T, H, hs] -> [B, H, T, hs]
        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        # Пропустите матрицы запроса и ключа через экземпляр rope, чтобы выполнить поворот.
        if self._rope is not None:
            # Применяем RoPE к Q и K (НЕ к V!)
            q = self._rope(q)  # [B, T, hs]
            k = self._rope(k)  # [B, T, hs]


        # Если cache пришел, то объединяем кэш и одну строку из ключа и значения. Это будут новые key и value  для последующих вычислений.
        # 5. Кэширование (для autoregressive generation)
        if cache is not None:
            k_cache, v_cache = cache
            k = torch.cat([k_cache, k], dim=2)  # Concat по seq_len (dim=2)
            v = torch.cat([v_cache, v], dim=2)


        # Перемножим матрицы запроса и ключа (транспонированную), чтобы вычислить матрицу внимания.
        # И разделить все значения в матрице внимания на корень из head_size.
        scores = q @ k.transpose(-2, -1) / (self._head_size ** 0.5)

        # Если cache пришел, то маску не накладываем. Иначе наложите на матрицу внимания треугольную маску, созданную при инициализации. Все скрытые значения должны быть приведены к минус бесконечности: float('-inf').
        if cache is None:
            scores = scores.masked_fill(
                ~self._tril_mask[:seq_len, :seq_len], float("-inf")
            )

        # Применить к матрице внимания (построчно) функцию Softmax.
        weights = F.softmax(scores, dim=-1)

        # Перемножим матрицу внимания и матрицу значения.
        x_out = weights @ v  # [B, T, hs]


        # Измените форму тензора на batch_size × seq_len × num_heads*head_size.
        # Transpose обратно и concatenate heads
        x_out = x_out.transpose(1, 2)  # [B, T_q, H, hs]
        x_out = x_out.contiguous()  # Важно для reshape!
        concatenated_attention = x_out.reshape(batch_size, seq_len, self._num_q_heads * self._head_size)


        # Пропустите получившийся тензор через последний линейный слой.
        # 3. Проецируем в пространство эмбеддингов
        projected_output = self._layer(concatenated_attention)


        # 4. Применяем dropout для регуляризации
        final_output = self._dropout(projected_output)

        if use_cache is True:
            return (final_output, (k, v))
        else:
            return (final_output, None)


In [35]:

# Параметры
batch_size = 2
seq_len = 10
emb_size = 512
num_q_heads = 8
head_size = 64
max_seq_len = 512

 # Создание модели
rope = RoPE(head_size=head_size, max_seq_len=max_seq_len)
mha = MultiQueryAttention(
    num_q_heads=num_q_heads,
    emb_size=emb_size,
    head_size=head_size,
    max_seq_len=max_seq_len,
    rope=rope,
    dropout=0.1,
)

 # Тест 1: Обычный forward pass
x = torch.randn(batch_size, seq_len, emb_size)
output, cache = mha(x, use_cache=False)
print(f"✅ Test 1 - Output shape: {output.shape}")  # [2, 10, 512]
assert output.shape == (batch_size, seq_len, emb_size)

 # Тест 2: С кэшированием
x1 = torch.randn(batch_size, 5, emb_size)
output1, cache1 = mha(x1, use_cache=True)
print(f"✅ Test 2 - First output shape: {output1.shape}")  # [2, 5, 512]

x2 = torch.randn(batch_size, 1, emb_size)
output2, cache2 = mha(x2, use_cache=True, cache=cache1)
print(f"✅ Test 2 - Second output shape: {output2.shape}")  # [2, 1, 512]

print("\n✅ Все тесты пройдены!")

✅ Test 1 - Output shape: torch.Size([2, 10, 512])
✅ Test 2 - First output shape: torch.Size([2, 5, 512])
✅ Test 2 - Second output shape: torch.Size([2, 1, 512])

✅ Все тесты пройдены!


Вот конвертированный Markdown для твоего HTML:

---

# GeGLU

<p align="center">
  <img src="https://ucarecdn.com/292c5458-2544-4e1f-a3f1-a981faeab39d/" alt="geglu" width="1000" height="586">
</p>

GeGLU — это гибридная функция активации.
По сути, это та же **SwiGLU**, которую мы реализовали в **Llama**, просто у неё в качестве базовой функции вместо **SiLU** (как в Llama) используется **GELU** (как в GPT-2).


**GeGLU (разработка)**

In [65]:
import math

class GELU(nn.Module):
    def __init__(self):
        super().__init__()
        self.sqrt_2_over_pi = torch.sqrt(torch.tensor(2.0) / math.pi)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return 0.5 * x * (1 + torch.tanh(
            self.sqrt_2_over_pi * (x + 0.044715 * torch.pow(x, 3))
        ))

class GeGLU(nn.Module):
    def __init__(self, emb_size: int, dropout: float = 0.1):
        super().__init__()

        self._gate = nn.Linear(emb_size, 4 * emb_size)
        self._up = nn.Linear(emb_size, 4 * emb_size)
        self._down = nn.Linear(4 * emb_size, emb_size)
        self._activation = GELU()
        self._dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor): # [batch_size × seq_len × emb_size].
        gate_out = self._gate(x)                          # [batch, seq, 4*emb]
        activation_out = self._activation(gate_out)       # [batch, seq, 4*emb]
        up_out = self._up(x)                              # [batch, seq, 4*emb]
        out = up_out * activation_out                     # поэлементное!
        out = self._down(out)                             # [batch, seq, emb]
        return self._dropout(out)


# Full Model

In [66]:
import torch
from torch import nn
from torch import Tensor
import torch.nn.functional as F
from math import sqrt


    
class RMSNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-6):
        super().__init__()
        self._eps = eps
        self._w = nn.Parameter(torch.ones(dim))
    
    def forward(self, x: torch.Tensor): # [batch_size × seq_len × emb_size]
        rms = (x.pow(2).mean(-1, keepdim=True) + self._eps) ** 0.5
        norm_x = x / rms
        return self._w * norm_x

class TokenEmbeddings(nn.Module):
    def __init__(self, vocab_size: int, emb_size: int):
        super().__init__()
        self._embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=emb_size
        )

    def forward(self, x: Tensor) -> Tensor:
        return self._embedding(x)

    @property
    def num_embeddings(self) -> int:
        return self._embedding.num_embeddings

    @property
    def embedding_dim(self) -> int:
        return self._embedding.embedding_dim



import torch
from torch import nn
from typing import Optional


class RoPE(nn.Module):

    def __init__(self, head_size: int, max_seq_len: int, base: int = 10_000):
        super().__init__()
        assert head_size % 2 == 0, "head_size должен быть четным"

        # Вычисление частот: θ_i = base^(-2i/d) для i ∈ [0, d/2-1]
        freqs = 1.0 / (base ** (2 * torch.arange(head_size // 2).float() / head_size))

        # Позиции от 0 до max_seq_len-1
        positions = torch.arange(max_seq_len).float()

        # Внешнее произведение: m * θ_i для всех позиций и частот
        freq_matrix = positions.unsqueeze(1) * freqs.unsqueeze(0)

        # Предвычисление матриц косинусов и синусов
        self.register_buffer("cos_matrix", torch.cos(freq_matrix))
        self.register_buffer("sin_matrix", torch.sin(freq_matrix))

    def forward(self, x: torch.Tensor) -> torch.Tensor: # [batch_size × seq_len × head_size] [batch_size × num_heads × seq_len × head_size]
        batch_size, num_heads, seq_len, head_size = x.shape

        # Берем нужную часть матриц и приводим к типу x
        cos = self.cos_matrix[:seq_len].to(x.dtype)  # [seq_len, head_size//2]
        sin = self.sin_matrix[:seq_len].to(x.dtype)  # [seq_len, head_size//2]

        # Явное изменение формы для broadcasting
        cos = cos.reshape(1, 1, seq_len, head_size // 2)
        sin = sin.reshape(1, 1, seq_len, head_size // 2)

        # Разделяем на четные и нечетные компоненты по ПОСЛЕДНЕМУ измерению
        x_even = x[..., 0::2]  # [batch_size, num_heads, seq_len, head_size//2]
        x_odd = x[..., 1::2]   # [batch_size, num_heads, seq_len, head_size//2]

        # Применяем поворот: q' = q * cos(mθ) + rotate(q) * sin(mθ)
        x_rotated_even = x_even * cos - x_odd * sin
        x_rotated_odd = x_even * sin + x_odd * cos

        # Объединяем обратно в исходную размерность
        x_rotated = torch.stack([x_rotated_even, x_rotated_odd], dim=-1)
        x_rotated = x_rotated.flatten(-2)  # [batch_size, seq_len, head_size]

        return x_rotated


class Decoder(nn.Module):
    def __init__(self, 
        num_q_heads: int,
        emb_size: int,
        head_size: int,
        max_seq_len: int,
        rope: RoPE,
        dropout: float = 0.1
    ):
        super().__init__()
        self._heads = MultiQueryAttention(
            num_q_heads=num_q_heads, 
            emb_size=emb_size, 
            head_size=head_size, 
            max_seq_len=max_seq_len,
            rope=rope,
            dropout=dropout
        )
        self._ff = GeGLU(emb_size=emb_size, dropout=dropout)
        self._norm1 = RMSNorm(emb_size)
        self._norm2 = RMSNorm(emb_size)

    def forward(self, x: torch.Tensor, mask: torch.Tensor = None, use_cache: bool = True, cache: list = None) -> torch.Tensor:
        norm1_out = self._norm1(x)
        attention, kv_caches = self._heads(norm1_out, mask, use_cache=use_cache, cache=cache)
        out = attention + x
        
        norm2_out = self._norm2(out)
        ffn_out = self._ff(norm2_out)

        if use_cache is True:
            return (ffn_out + out, kv_caches)
        else:
            return (ffn_out + out, None)



from torch import nn
import torch
import torch.nn.functional as F

class Gemma(nn.Module):
    def __init__(self,
        vocab_size: int,
        max_seq_len: int,
        emb_size: int,
        num_q_heads: int,
        head_size: int,
        num_layers: int,
        dropout: float = 0.1,
        device: str = 'cpu'
    ):
        super().__init__()
        self._vocab_size = vocab_size
        self._max_seq_len = max_seq_len
        self._emb_size = emb_size
        self._num_q_heads = num_q_heads
        self._head_size = head_size
        self._num_layers = num_layers
        self._dropout = dropout
        self._device = device
        
        self.validation_loss = None

        # Инициализация слоев
        self._token_embeddings = TokenEmbeddings(
            vocab_size=vocab_size, 
            emb_size=emb_size
        )
        self._position_embeddings = RoPE(
            head_size=head_size,
            max_seq_len=max_seq_len
        )
        #self._position_embeddings = PositionalEmbeddings(
        #    max_seq_len=max_seq_len, 
        #    emb_size=emb_size
        #)
        self._dropout = nn.Dropout(dropout)
        self._decoders = nn.ModuleList([Decoder(
            num_q_heads=num_q_heads,
            emb_size=emb_size,
            head_size=head_size,
            max_seq_len=max_seq_len,
            rope=self._position_embeddings,
            dropout=dropout 
        ) for _ in range(num_layers)])
        self._norm = RMSNorm(emb_size)
        self._linear = nn.Linear(emb_size, vocab_size)

    def forward(self, x: torch.Tensor, use_cache: bool = True, cache: list = None) -> tuple:
        # Проверка длины последовательности (только при отсутствии кэша)
        if cache is None and x.size(1) > self._max_seq_len:
            raise ValueError(f"Длина последовательности {x.size(1)} превышает максимальную {self.max_seq_len}")
        
        # Эмбеддинги токенов и позиций
        tok_out = self._token_embeddings(x)  # [batch, seq_len, emb_size]
       #pos_out = self._position_embeddings(x)  # [batch, seq_len, emb_size]
        
        # Комбинирование
        out = self._dropout(tok_out)  # [batch, seq_len, emb_size]
        
        # Стек декодеров с передачей кэша
        new_cache = []
        for i, decoder in enumerate(self._decoders):
            decoder_cache = cache[i] if cache is not None else None
            decoder_result = decoder(out, use_cache=use_cache, cache=decoder_cache)

            # Извлекаем результат из кортежа
            if use_cache:
                out, decoder_new_cache = decoder_result
                new_cache.append(decoder_new_cache)
            else:
                out = decoder_result[0]

        out = self._norm(out)
        logits = self._linear(out)
            
        # Возвращаем результат с учетом use_cache
        if use_cache:
            return (logits, new_cache)
        else:
            return (logits, None)

    def generate(self,
        x: torch.Tensor, 
        max_new_tokens: int, 
        do_sample: bool,
        temperature: float = 1.0,
        top_k: int = None,
        top_p: float = None,
        use_cache: bool = True
    ) -> torch.Tensor:
        cache = None

        for _ in range(max_new_tokens):
            if use_cache and cache is not None:
                # Используем кэш - передаем только последний токен
                x_input = x[:, -1:]  # [batch_size, 1]
            else:
                # Первая итерация или кэш отключен - передаем всю последовательность
                x_input = x
            
            # Прямой проход с кэшем
            logits, new_cache = self.forward(x_input, use_cache=use_cache, cache=cache)
            
            # Обновляем кэш для следующей итерации
            if use_cache:
                cache = new_cache

            last_logits = logits[:, -1, :]  # [batch_size, vocab_size]

            # Масштабируем логиты температурой
            if temperature > 0:
                logits_scaled = last_logits / temperature
            else:
                logits_scaled = last_logits

            if do_sample == True and top_k != None:
                _, topk_indices = torch.topk(logits_scaled, top_k, dim=-1)

                # # Заменим все НЕ top-k логиты на -inf
                masked_logits = logits_scaled.clone()
                vocab_size = logits_scaled.size(-1)

                # создаём маску: 1, если токен НЕ в topk_indices
                mask = torch.ones_like(logits_scaled, dtype=torch.uint8)
                mask.scatter_(1, topk_indices, 0)  # 0 там, где top-k индексы
                masked_logits[mask.byte()] = float('-inf')

                logits_scaled = masked_logits

            if do_sample == True and top_p != None:
                # 1. Применим softmax, чтобы получить вероятности:
                probs = F.softmax(logits_scaled, dim=-1)  # [B, vocab_size]
                # 2. Отсортируем токены по убыванию вероятностей:
                sorted_probs, sorted_indices = torch.sort(probs, descending=True, dim=-1)
                # 3. Посчитаем кумулятивную сумму вероятностей:
                cum_probs = torch.cumsum(sorted_probs, dim=-1)  # [B, vocab_size]
                # 4. Определим маску: оставить токены, пока сумма < top_p
                sorted_mask = (cum_probs <= top_p).byte()  # [B, vocab_size]
                # Гарантируем, что хотя бы первый токен останется
                sorted_mask[:, 0] = 1
                # 5. Преобразуем маску обратно в оригинальный порядок:
                # Создаём полную маску из 0
                mask = torch.zeros_like(probs, dtype=torch.uint8)
                # Устанавливаем 1 в местах нужных токенов
                mask.scatter_(dim=1, index=sorted_indices, src=sorted_mask)
                # 6. Зануляем логиты токенов вне топ-p:
                logits_scaled[~mask] = float('-inf')

            # 4. Применяем Softmax
            probs = F.softmax(logits_scaled, dim=-1)  # [batch_size, vocab_size]


            if do_sample == True:
                # 5. Если do_sample равен True, то отбираем токен случайно с помощью torch.multinomial
                next_token = torch.multinomial(probs, num_samples=1)  # [batch_size, 1]
            else:
                # 5. Если do_sample равен False, то выбираем токен с максимальной вероятностью
                next_token = torch.argmax(probs, dim=-1, keepdim=True)  # [batch_size, 1]
            
            # 6. Добавляем его к последовательности
            x = torch.cat([x, next_token], dim=1)  # [batch_size, seq_len+1]
        return x

    def save(self, path):
        torch.save({
            'model_state_dict': self.state_dict(),
            'vocab_size': self._vocab_size,
            'max_seq_len': self._max_seq_len,
            'emb_size': self._emb_size,
            'num_heads': self._num_heads,
            'head_size': self._head_size,
            'num_layers': self._num_layers
        }, path)

    @classmethod
    def load(cls, path, device):
        checkpoint = torch.load(path, map_location=device)
        model = cls(
            vocab_size=checkpoint['vocab_size'],
            max_seq_len=checkpoint['max_seq_len'],
            emb_size=checkpoint['emb_size'],
            num_heads=checkpoint['num_heads'],
            head_size=checkpoint['head_size'],
            num_layers=checkpoint['num_layers']
        )
        model.load_state_dict(checkpoint['model_state_dict'])
        model.to(device)
        return model

    @property
    def max_seq_len(self) -> int:
        return self._max_seq_len






## 2. Обучение Gemma

Gemma обучается в два этапа:

- 1️⃣ **Предобучение (Unsupervised Pretraining)**  
- 2️⃣ **Дообучение (Supervised Fine-Tuning)**



### 5.1 Предобучение

На первом этапе модель обучается без разметки: она получает большой корпус текстов и учится **предсказывать следующий токен** по предыдущим.

Функция потерь:
$$
L = - \sum_{t=1}^{T} \log P(x_t | x_1, x_2, ..., x_{t-1})
$$

Таким образом, модель учится строить вероятностную модель языка, "угадывая" продолжение текста.


Во время **предобучения** Mistral учится **предсказывать следующий токен** (language modeling task).  
Формально:  
$$ 
P(x_t ,|, x_1, x_2, \dots, x_{t-1})  
$$ 
То есть, если на вход подаётся предложение `"I love deep"`, модель должна предсказать `"learning"`.


### ✅ 5.1.1 Подготовка данных

Создадим **датасет** на основе BPE-токенизатора:

**BPE Tokenizator**

In [67]:
class BPE:
    def __init__(self, vocab_size: int):
        self.vocab_size = vocab_size
        self.id2token = {}
        self.token2id = {}

    def fit(self, text: str):
        # 1. Получаем уникальные токены (символы)
        unique_tokens = sorted(set(text))
        tokens = unique_tokens.copy()

        # 2. Разбиваем текст на токены-символы
        sequence = list(text)

        # 3. Объединяем токены до достижения нужного размера словаря
        while len(tokens) < self.vocab_size:
            #print(f'len={len(tokens)} < {self.vocab_size}')
            # Считаем частоты пар
            pair_freq = {}
            for i in range(len(sequence) - 1):
                pair = (sequence[i], sequence[i + 1])
                #print(f'pair = {pair}')
                if pair not in pair_freq:
                    pair_freq[pair] = 0
                pair_freq[pair] += 1


            #print(f'pair_freq = {pair_freq}')  
            if not pair_freq:
                break  # нет пар — выходим

            #for x in pair_freq.items():
            #    self.debug(x, sequence)

            # Находим самую частую пару (в случае равенства — та, что встретилась первой)
            most_frequent_pair = max(pair_freq.items(), key=lambda x: (x[1], -self._pair_first_index(sequence, x[0])))[0]
            #print(most_frequent_pair)
            # Создаем новый токен
            new_token = most_frequent_pair[0] + most_frequent_pair[1]
            #print(f"new token={new_token}")
            tokens.append(new_token)
            #print(f"tokens={tokens}")

            i = 0
            new_sequence = []

            while i < len(sequence):
                if i < len(sequence) - 1 and (sequence[i], sequence[i + 1]) == most_frequent_pair:
                    new_sequence.append(new_token)
                    i += 2  # пропускаем два символа — заменённую пару
                else:
                    new_sequence.append(sequence[i])
                    i += 1
            sequence = new_sequence
            #break
        
        # 4. Создаем словари
        self.vocab = tokens.copy()
        self.token2id = dict(zip(tokens, range(self.vocab_size)))
        self.id2token = dict(zip(range(self.vocab_size), tokens))

    def _pair_first_index(self, sequence, pair):
        for i in range(len(sequence) - 1):
            if (sequence[i], sequence[i + 1]) == pair:
                return i
        return float('inf')  # если пара не найдена (в теории не должно случиться)


    def encode(self, text: str):
        # 1. Разбиваем текст на токены-символы
        sequence = list(text)
        # 2. Инициализация пустого списка токенов
        tokens = []
        # 3. Установить i = 0
        i = 0
        while i < len(text):
            # 3.1 Найти все токены в словаре, начинающиеся с text[i]
            start_char = text[i]
            result = [token for token in self.vocab if token.startswith(start_char)]
            # 3.2 Выбрать самый длинный подходящий токен
            find_token = self._find_max_matching_token(text[i:], result)
            if find_token is None:
                # Обработка неизвестного символа
                tokens.append(text[i])  # Добавляем сам символ как токен
                i += 1
            else:
                # 3.3 Добавить токен в результат
                tokens.append(find_token)
                # 3.4 Увеличить i на длину токена
                i += len(find_token)

        # 4. Заменить токены на их ID
        return self._tokens_to_ids(tokens)

    def _find_max_matching_token(self, text: str, tokens: list):
        """Находит самый длинный токен из списка, с которого начинается текст"""
        matching = [token for token in tokens if text.startswith(token)]
        return max(matching, key=len) if matching else None

    def _tokens_to_ids(self, tokens):
        """Конвертирует список токенов в их ID с обработкой неизвестных токенов"""
        ids = []
        for token in tokens:
            if token in self.token2id:
                ids.append(self.token2id[token])
            else:
                ids.append(0)  # Специальное значение
        return ids


    def decode(self, ids: list) -> str:
        return ''.join(self._ids_to_tokens(ids))

    def _ids_to_tokens(self, ids: list) -> list:
        """Конвертирует список Ids в их tokens"""
        tokens = []
        for id in ids:
            if id in self.id2token:
                tokens.append(self.id2token[id])
            else:
                tokens.append('')  # Специальное значение
        return tokens


    def save(self, filename):
        with open(filename, 'wb') as f:
            dill.dump(self, f)
        print(f"Объект сохранён в {filename}")


    @classmethod
    def load(cls, filename):
        with open(filename, 'rb') as f:
            obj = dill.load(f)
                
        print(f"Объект загружен из {filename}")
        return obj

In [68]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDataset(Dataset):
    def __init__(self, text: str, bpe: BPE, block_size: int):
        self.bpe = bpe
        self.block_size = block_size
        self.data = bpe.encode(text)
        
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx:idx+self.block_size], dtype=torch.long)
        y = torch.tensor(self.data[idx+1:idx+self.block_size+1], dtype=torch.long)
        return x, y

### ✅ 5.1.2 Цикл обучения

Для обучения создадим функцию:

In [69]:
import torch.nn.functional as F
from torch import optim

def train_gemma(model, dataset, epochs=5, batch_size=32, lr=3e-4, device='cpu'):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = optim.AdamW(model.parameters(), lr=lr)

    model.to(device)
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)

            # Прямой проход
            logits, _ = model(x, use_cache=False)  # [B, T, vocab_size]

            # Перестроим выход под CrossEntropy
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1))

            # Обратное распространение
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

    return model

### ✅ 5.1.3 Пример запуска


**🧠 Конфигурация Gemma Mini**


| Параметр        | Значение | Описание                                      |
| --------------- | -------- | --------------------------------------------- |
| **vocab_size**  | `50257`  | Размер словаря (BPE токенизатор OpenAI)       |
| **max_seq_len** | `512`   | Максимальная длина входной последовательности |
| **emb_size**    | `256`    | Размер эмбеддингов (векторное пространство)   |
| **num_heads**   | `4`     | Количество голов в multi-head attention       |
| **head_size**   | `64`     | Размерность одной головы внимания (768 / 12)  |
| **num_layers**  | `4`     | Количество блоков (декодеров)                 |
| **dropout**     | `0.1`    | Вероятность дропаута                          |


In [70]:
# 1. Исходный текст
text = "Deep learning is amazing. Transformers changed the world. Attention is all you need. GPT models revolutionized NLP."

# 2. Обучаем токенизатор
bpe = BPE(vocab_size=100)
bpe.fit(text)

# 3. Создаем датасет
dataset = GPTDataset(text, bpe, block_size=8)
print(f"Dataset length: {len(dataset)}")

# 4. Инициализируем модель
model = Gemma(
    vocab_size=len(bpe.vocab),  # размер словаря BPE
    max_seq_len=512,           # GPT-2 использует контекст в 512 токена
    emb_size=256,               # размер эмбеддингов
    num_q_heads=4,               # количество голов внимания
    head_size=64,               # размер каждой головы (256 / 4)
    num_layers=4,              # количество блоков Transformer
    dropout=0.1                 # стандартный dropout GPT-2
)

# 5. Обучаем
train_gemma(model, dataset, epochs=100, batch_size=4)

Dataset length: 20
Epoch 1/100, Loss: 3.8178
Epoch 2/100, Loss: 1.5683
Epoch 3/100, Loss: 0.6454
Epoch 4/100, Loss: 0.3353
Epoch 5/100, Loss: 0.2306
Epoch 6/100, Loss: 0.1581
Epoch 7/100, Loss: 0.1253
Epoch 8/100, Loss: 0.1063
Epoch 9/100, Loss: 0.0923
Epoch 10/100, Loss: 0.0909
Epoch 11/100, Loss: 0.0761
Epoch 12/100, Loss: 0.0932
Epoch 13/100, Loss: 0.0775
Epoch 14/100, Loss: 0.0797
Epoch 15/100, Loss: 0.0623
Epoch 16/100, Loss: 0.0795
Epoch 17/100, Loss: 0.0703
Epoch 18/100, Loss: 0.0581
Epoch 19/100, Loss: 0.0613
Epoch 20/100, Loss: 0.0660
Epoch 21/100, Loss: 0.0731
Epoch 22/100, Loss: 0.0644
Epoch 23/100, Loss: 0.0602
Epoch 24/100, Loss: 0.0557
Epoch 25/100, Loss: 0.0595
Epoch 26/100, Loss: 0.0688
Epoch 27/100, Loss: 0.0545
Epoch 28/100, Loss: 0.0561
Epoch 29/100, Loss: 0.0581
Epoch 30/100, Loss: 0.0627
Epoch 31/100, Loss: 0.0555
Epoch 32/100, Loss: 0.0538
Epoch 33/100, Loss: 0.0531
Epoch 34/100, Loss: 0.0535
Epoch 35/100, Loss: 0.0474
Epoch 36/100, Loss: 0.0516
Epoch 37/100, Loss

Gemma(
  (_token_embeddings): TokenEmbeddings(
    (_embedding): Embedding(100, 256)
  )
  (_position_embeddings): RoPE()
  (_dropout): Dropout(p=0.1, inplace=False)
  (_decoders): ModuleList(
    (0-3): 4 x Decoder(
      (_heads): MultiQueryAttention(
        (_rope): RoPE()
        (_q): Linear(in_features=256, out_features=256, bias=True)
        (_k): Linear(in_features=256, out_features=64, bias=True)
        (_v): Linear(in_features=256, out_features=64, bias=True)
        (_layer): Linear(in_features=256, out_features=256, bias=True)
        (_dropout): Dropout(p=0.1, inplace=False)
      )
      (_ff): GeGLU(
        (_gate): Linear(in_features=256, out_features=1024, bias=True)
        (_up): Linear(in_features=256, out_features=1024, bias=True)
        (_down): Linear(in_features=1024, out_features=256, bias=True)
        (_activation): GELU()
        (_dropout): Dropout(p=0.1, inplace=False)
      )
      (_norm1): RMSNorm()
      (_norm2): RMSNorm()
    )
  )
  (_norm): RM


---

### 5.2 Дообучение

После предобучения Gemma уже знает структуру и грамматику языка.  
На втором этапе она дообучается на конкретных задачах (например, классификация, QA) с помощью размеченных данных.

Технически это почти то же обучение, только:

- Загружаем модель с уже обученными весами.
- Используем новые данные.
- Можно уменьшить скорость обучения.
- Иногда замораживают часть слоёв (например, эмбеддинги).


In [71]:
def fine_tune_gemma(model, dataset, epochs=3, batch_size=16, lr=1e-5, device='cpu', freeze_embeddings=True):
    if freeze_embeddings:
        for param in model._token_embeddings.parameters():
            param.requires_grad = False
        for param in model._position_embeddings.parameters():
            param.requires_grad = False

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

    model.to(device)
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            logits, _ = model(x, use_cache=False)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Fine-tune Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}")

In [72]:
# Например, мы хотим дообучить модель на стиле коротких технических фраз
fine_tune_text = """
Transformers revolutionize NLP.
Deep learning enables self-attention.
GPT generates text autoregressively.
"""

dataset = GPTDataset(fine_tune_text, bpe, block_size=8)


# Запуск дообучения
fine_tune_gemma(model, dataset, epochs=10, batch_size=4, lr=1e-4)

Fine-tune Epoch 1/10, Loss: 4.9095
Fine-tune Epoch 2/10, Loss: 2.8684
Fine-tune Epoch 3/10, Loss: 1.7589
Fine-tune Epoch 4/10, Loss: 1.3044
Fine-tune Epoch 5/10, Loss: 1.0614
Fine-tune Epoch 6/10, Loss: 0.8326
Fine-tune Epoch 7/10, Loss: 0.6908
Fine-tune Epoch 8/10, Loss: 0.5926
Fine-tune Epoch 9/10, Loss: 0.5082
Fine-tune Epoch 10/10, Loss: 0.4758


## 📝 6. Генерация текста после обучения

In [73]:
def generate_text(model, bpe, prompt: str, max_new_tokens=20, device='cpu'):
    model.eval()
    ids = torch.tensor([bpe.encode(prompt)], dtype=torch.long).to(device)
    out = model.generate(ids, max_new_tokens=max_new_tokens, do_sample=True)
    text = bpe.decode(out[0].tolist())
    return text

In [74]:
print(generate_text(model, bpe, "Deep learning", max_new_tokens=20))

Deep learningenena lf lenenssssf 
